In [67]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
import numpy as np, random, pickle
from itertools import cycle
from sklearn.neighbors import LocalOutlierFactor
%matplotlib inline
from sklearn.feature_selection import mutual_info_regression, mutual_info_classif,SelectKBest,f_classif
import scipy.stats
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from typing import List
import seaborn as sns; sns.set()

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Load Data

In [73]:
#df = pd.read_csv('/content/drive/MyDrive/Masters Project/Gun_Violence_Record_Update.csv')
df = pd.read_csv('/content/drive/MyDrive/Masters_Final/Gun_Violence_Record_Update_Final (1).csv')

#Save Selected Features

In [74]:
df= df[['State Code', 'Region', 'Urban/Suburban/Rural', 'Age', 'Race',
       'Religion', 'Education', 'School Performance', 'Birth Order',
       'Number of Siblings', 'Relationship Status', 'Children',
       'Employment Status', 'Employment Type\xa0', 'Military Branch',
       'Community Involvement', 'Part I Crimes', 'Part II Crimes',
       'Domestic Abuse Specified', 'Childhood SES',
       'Recent or Ongoing Stressor', 'Timeline of Signs of Crisis',
       'Substance Use', 'Known Prejudices\xa0', 'Leakage How', 'Leakage Who\xa0',
       'Leakage Specific/Nonspecific ', 'Criminal Sentence', 'label']]

#Preprocess Data

In [75]:
df.replace(r'^\s+$', np.nan, regex=True,  inplace=True)
#df['Kidpping or Hostage Situation'].replace('S', 1,  inplace=True)
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.fillna(-999999, inplace=True)

In [76]:
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=100, test_size =0.3)

In [77]:
X = pd.DataFrame.from_records(X_train)
X.columns =[['State Code', 'Region', 'Urban/Suburban/Rural', 'Age', 'Race',
       'Religion', 'Education', 'School Performance', 'Birth Order',
       'Number of Siblings', 'Relationship Status', 'Children',
       'Employment Status', 'Employment Type\xa0', 'Military Branch',
       'Community Involvement', 'Part I Crimes', 'Part II Crimes',
       'Domestic Abuse Specified', 'Childhood SES',
       'Recent or Ongoing Stressor', 'Timeline of Signs of Crisis',
       'Substance Use', 'Known Prejudices\xa0', 'Leakage How', 'Leakage Who\xa0',
       'Leakage Specific/Nonspecific ', 'Criminal Sentence']]



#Calculate Euclidean distance

In [78]:
## Euclidean distance
def dst(v1, v2):
  #print(f'v1: {v1}\nv2: {v2}')
  return np.sum([(xi-yi)**2 for xi, yi in zip(v1,v2)])


#Compute Centriod

In [79]:
def compute_centroid(cluster):
  #print(f'cluster: \n{np.array(cluster).shape}')
  return np.mean(cluster, axis=1)

#Creating the Kmeans model

In [80]:
from sklearn.metrics import cluster
def k_means(X, k:int):
  rnd_centers = np.random.choice(range(len(X)), k, replace=False)
  centroids = [X[p] for p in rnd_centers]
  
  while True:
    clusters = [[] for i in range(len(centroids))]
    for x in X:
      proximities = [dst(x,c) for c in centroids]
      closest_cen = np.argmin(proximities)
      clusters[closest_cen].append(x)
    
    #for i, cluster in enumerate(clusters):
     # print(f'cluster-{i} - {len(cluster)} -> {cluster}' )
    #break
    new_centroids = [compute_centroid(c) for c in clusters if len(c) > 0]
    #for i, c in enumerate(new_centroids):
      #print(f'centroid - {i+1}: {c}')


    
    if np.array_equal(new_centroids, centroids):
      break
    centroids = new_centroids
    #print(f'centroids: {len(centroids)}\n{centroids}')
  return centroids, clusters

1/m$\sum_{i=1}^{m}{||x_i-C_k||_2^2}$

In [81]:
def plot_clusters(X):
  X= np.array(X)
  plt.scatter(X[:,0], X[:,1], s=100, cmap='viridis')

In [82]:
#X = df
#type(df)
data_points = np.array(X.values.tolist())
data_points.shape

(126, 28)

In [83]:
type(data_points)

numpy.ndarray

#Hash Data points

In [236]:
data_hashes = {hash(tuple(x)): x for x in data_points}
print(data_hashes)
with open('data_hashes.pickle', 'wb') as f:
    pickle.dump(data_hashes, f)

{1969188865212739700: array([ 1.20000e+01,  3.00000e+00,  2.00000e+00,  5.30000e+01,
        0.00000e+00, -9.99999e+05, -9.99999e+05, -9.99999e+05,
       -9.99999e+05, -9.99999e+05,  3.00000e+00,  1.00000e+00,
        0.00000e+00, -9.99999e+05, -9.99999e+05,  1.00000e+00,
        0.00000e+00,  0.00000e+00,  0.00000e+00, -9.99999e+05,
        3.00000e+00,  1.00000e+00,  1.00000e+00,  0.00000e+00,
        0.00000e+00,  0.00000e+00, -9.99999e+05,  0.00000e+00]), -2512731333725551770: array([ 3.70000e+01,  3.00000e+00,  2.00000e+00,  2.60000e+01,
        0.00000e+00,  0.00000e+00, -9.99999e+05,  2.00000e+00,
       -9.99999e+05, -9.99999e+05,  3.00000e+00,  0.00000e+00,
        0.00000e+00,  0.00000e+00,  1.00000e+00,  0.00000e+00,
        0.00000e+00,  0.00000e+00,  4.00000e+00,  1.00000e+00,
        3.00000e+00,  1.00000e+00,  4.00000e+00,  0.00000e+00,
        1.00000e+00,  9.00000e+00,  1.00000e+00,  2.00000e+00]), 4143861648008690851: array([ 4.30000e+01,  0.00000e+00,  2.00000e+00, 

#Training Kmeans model

In [88]:
%%time
centroids, clusters = k_means(data_points, 4)

CPU times: user 42.9 ms, sys: 2.17 ms, total: 45.1 ms
Wall time: 41 ms


In [89]:
# Save model
with open('kmeans_model.pickle', 'wb') as f:
    pickle.dump(clusters, f)

In [90]:
np.array(clusters[0]).shape

(126, 28)

#Scatter plot of clusters

In [91]:
plot_clusters(clusters[0])

#Calculate Scores

In [92]:
def sim_score(vec1, vec2):
  if len(vec1) != len(vec2):
    #print(len(vec1) , print(vec2))
    raise ValueError(f'vec1 & vec2 must be of similar dimensions or padded')
  x = np.sum([v1*v2 for v1,v2 in zip(vec1, vec2)])
  y = np.linalg.norm(vec1, ord=2)*np.linalg.norm(vec2, ord=2)

  return x/y if y > 0 else 0

#Predict new datapoint for Kmeans- one at a time

In [102]:
def predict(data):
  closest_idx = np.argmax([sim_score(data, point) for point in clusters[0]])
  print(f'{closest_idx}')
  closest_idx_key = hash(tuple(clusters[0][closest_idx]))
  print(f'{closest_idx_key}')
  return sim_score(data, clusters[0][closest_idx]), data_hashes[closest_idx_key]

#Predict 3 max datapoints for kmeans

In [151]:
def predict3(data, clusters):
  scores= []
  keys = []
  output = []
  for point in clusters[0]:
    closest_idx = sim_score(data, point)
    scores.append(closest_idx)

  res = sorted(range(len(scores)), key=lambda sub: scores[sub])[-3:]
  print(res)

  for i in res:
    keys.append(hash(tuple(clusters[0][i])))

  for  closest_idx_key in keys:
    output.append((data_hashes[closest_idx_key]).tolist())

  for i in range(len(res)):
    test =(sim_score(data, clusters[0][res[i]]))
    output[i].append(round(test, 4))

  return output

#Get original values

In [153]:
def getOriginalData(arr):
  selected_columns =['State Code', 'Region', 'Urban/Suburban/Rural', 'Age', 'Race',
       'Religion', 'Education', 'School Performance', 'Birth Order',
       'Number of Siblings', 'Relationship Status', 'Children',
       'Employment Status', 'Employment Type\xa0', 'Military Branch',
       'Community Involvement', 'Part I Crimes', 'Part II Crimes',
       'Domestic Abuse Specified', 'Childhood SES',
       'Recent or Ongoing Stressor', 'Timeline of Signs of Crisis',
       'Substance Use', 'Known Prejudices\xa0', 'Leakage How', 'Leakage Who\xa0',
       'Leakage Specific/Nonspecific\xa0', 'Crimil Sentence']
  matched_index = None
  df_original = pd.read_csv('/content/drive/MyDrive/Masters Project/Gun_Violence_Record__.csv')
  df = pd.read_csv('/content/drive/MyDrive/Masters Project/Gun_Violence_Record_Update.csv')
  df= df[selected_columns]
  df_original= df_original[['State Code', 'Region', 'Urban/Suburban/Rural', 'Age', 'Race',
       'Religion', 'Education', 'School Performance', 'Birth Order',
       'Number of Siblings', 'Relationship Status', 'Children',
       'Employment Status', 'Employment Type\xa0', 'Military Branch',
       'Community Involvement', 'Part I Crimes', 'Part II Crimes',
       'Domestic Abuse Specified', 'Childhood SES',
       'Recent or Ongoing Stressor', 'Timeline of Signs of Crisis',
       'Substance Use', 'Known Prejudices\xa0', 'Leakage How', 'Leakage Who\xa0',
       'Leakage Specific/Nonspecific ', 'Criminal Sentence']]
  df.replace(r'^\s+$', np.nan, regex=True,  inplace=True)
  df.replace([np.inf, -np.inf], np.nan, inplace=True)
  df.fillna(999, inplace=True)
  arr= pd.DataFrame([arr], columns= selected_columns)
  arr=arr.squeeze()
  for index, row in df.iterrows():
    result = row.equals(arr)
    if result:
      return df_original.iloc[index].tolist()
  return matched_index

In [113]:
scores = []
count = 0
output = []

res = sorted(range(len(scores)), key = lambda sub: scores[sub])[-3:]
for i in res:
  output.append(X[i].tolist())
print(output)
for i in range(len(res)):
  output[i].append(round(scores[res[i]], 2))
print(output)


[]
[]


#Test Model

In [239]:
predict([6.0, 3.0, 0.0, 24.0, 0.0, 1.0, 4.0, 2.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 3.0, 2.0, 3.0, 0.0, 1.0, 7.0, 1.0, 2.0 ])
    #[18.,  0.,0, 5.,  1.,  1.,  2.,  1.,  2.,  4.,  0.,  0.,  0.,  0.,
         #1.,  2.,  1.,  0.,  0.,  0.,  0.,  0.,  2.,  50.,  2., 0.,  1.,0])

86
395214719517510599


(0.8593550002024863,
 array([13.,  1.,  0., 27.,  0.,  0.,  4.,  2.,  3.,  1.,  1.,  0.,  0.,
         0.,  0.,  3.,  0., 13.,  7.,  1.,  2.,  3.,  3.,  1.,  4.,  3.,
         0.,  0.]))

In [230]:
output = predict3([18.,  0.,0, 5.,  1.,  1.,  2.,  1.,  2.,  4.,  0.,  0.,  0.,  0.,
         1.,  2.,  1.,  0.,  0.,  0.,  0.,  0.,  2.,  50.,  2., 0.,  1.,0])
print(output)

[86, 55, 125]
[[13.0, 1.0, 0.0, 27.0, 0.0, 0.0, 4.0, 2.0, 3.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 13.0, 7.0, 1.0, 2.0, 3.0, 3.0, 1.0, 4.0, 3.0, 0.0, 0.0, 0.2458], [43.0, 0.0, 0.0, 39.0, 4.0, 2.0, 4.0, 2.0, 1.0, 2.0, 0.0, 0.0, 1.0, 1.0, 0.0, 2.0, 0.0, 0.0, 0.0, 1.0, 2.0, 2.0, 0.0, 4.0, 1.0, 10.0, 0.0, 1.0, 0.375], [18.0, 0.0, 0.0, 23.0, 1.0, 1.0, 2.0, 1.0, 2.0, 4.0, 0.0, 0.0, 0.0, 0.0, 1.0, 2.0, 1.0, 9.0, 0.0, 1.0, 6.0, 3.0, 2.0, 5.0, 2.0, 10.0, 1.0, 0.0, 0.4014]]


In [234]:
from pandas.io.pytables import is_string_dtype
original_data=[]
all_data=[]
for i in output:
  #print(i)
  all_data = (getOriginalData(i[:-1]))
  all_data= [(x.split(',')) if isinstance(x, str) else x for x in all_data ]
  #print(type(all_data))
  for j in range(len(all_data)):
    if isinstance(all_data[j], list):
      all_data[j]= [float(x) for x in all_data[j]]
    else:
      all_data[j] = float(all_data[j])
      
  all_data.append(round(i[-1],4))
  original_data.append(all_data)


print(original_data)


<class 'list'>
<class 'list'>
<class 'list'>
[[13.0, 1.0, 0.0, 27.0, 0.0, 0.0, 4.0, 2.0, 3.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 3.0, [0.0], [4.0, 9.0], [3.0, 4.0], 1.0, [2.0], 3.0, [3.0], [1.0], [3.0], [2.0], [0.0], 0.0, 0.2458], [43.0, 0.0, 0.0, 39.0, 4.0, 2.0, 4.0, 2.0, 1.0, 2.0, 0.0, 0.0, 1.0, 1.0, 0.0, 2.0, [0.0], [0.0], [0.0], 1.0, [2.0], 2.0, [0.0], [4.0], [0.0], [9.0], [0.0], 1.0, 0.375], [18.0, 0.0, 0.0, 23.0, 1.0, 1.0, 2.0, 1.0, 2.0, 4.0, 0.0, 0.0, 0.0, 0.0, 1.0, 2.0, [1.0], [9.0], [0.0], 1.0, [6.0], 3.0, [2.0], [1.0, 4.0], [1.0, 1.0], [9.0, 1.0], [1.0, 0.0], 0.0, 0.4014]]


In [ ]:
X_validate = pd.DataFrame.from_records(X_test)

#Predict all test data for k-means

In [ ]:
#%%timeit
for i in X_test:
  result = predict(i)
  print(round(result[0], 4))

#Predict all Test data using LOF

In [30]:
lof = LocalOutlierFactor(n_neighbors = 20, novelty = True)
lof.fit(X_train)
y_pred = lof.predict(X_test)
test_scores = lof.decision_function(X_test)

print(f'test scores: {test_scores}')
print(y_pred)
print(lof)


test scores: [ 0.36697643  0.52386506  0.2079705   0.486679    0.36685146  0.42059842
  0.46844198  0.40427675  0.21316342  0.44823018  0.28132694  0.44823018
  0.37403302  0.30524715  0.51231795  0.04007485  0.38916636  0.07586865
  0.45410844  0.51006164  0.52223727  0.35075504  0.52718346  0.4701225
  0.4265954   0.30866812  0.33554946  0.41393389  0.47202172  0.51333991
  0.43987723  0.41236043  0.41090407  0.38916636  0.41327652  0.41834496
  0.46148828  0.16217564  0.49723153  0.48122631  0.35939642  0.56493512
  0.34649946  0.37373533  0.45877048  0.50464151 -0.35120341  0.41848735
  0.24362582  0.50464118  0.49653713  0.51006164  0.48774322  0.47918781
  0.29776659]
[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 -1  1
  1  1  1  1  1  1  1]
LocalOutlierFactor(novelty=True)


In [33]:
y_pred[y_pred == -1] = 0
print(f'LOF accuracy score: {accuracy_score(y_test, y_pred )}')
print(f'LOF precision score: {precision_score(y_test, y_pred)}')

LOF accuracy score: 0.9818181818181818
LOF precision score: 1.0


In [34]:
sim_index = lof.kneighbors(X_test,n_neighbors = 1,return_distance= False)

In [ ]:

#%%timeit
print("~~~~Scores ~~~test Data ~~~ Historical Data ~~~ Similarity Score~~~~ ")
similarity_score = []
for i,v in enumerate(sim_index):
  train_data = X_train[[v[0]]]
  test_data = X_test[[i]]
  score = sim_score(test_data, train_data)
  print(f'~~~~{round(test_scores[i], 2)}~~~~~~ {i} ~~~~~~~~~~~~~~~~ {v[0]}~~~~~~~~~~~~~{round(score, 4)}')


#Training iForest model

In [38]:
from sklearn.ensemble import IsolationForest
from sklearn.metrics import make_scorer, f1_score
from sklearn import model_selection
from sklearn.metrics import classification_report, accuracy_score, precision_score

In [41]:

%%timeit
clf = IsolationForest(random_state=0)
clf.fit(X_train)


135 ms ± 2.41 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


#Testing iForest model

In [44]:
clf = IsolationForest(random_state=0)
clf.fit(X_train)
scores_pred = clf.decision_function(X_test)
y_pred = clf.predict(X_test)
print(y_pred)

[-1  1  1  1 -1  1  1  1 -1  1  1  1  1 -1  1 -1 -1 -1 -1  1  1  1  1 -1
  1  1 -1  1  1 -1  1 -1  1  1  1 -1  1  1  1 -1 -1  1  1 -1 -1 -1 -1  1
 -1  1  1  1  1 -1  1]


In [45]:
y_pred[y_pred == -1] = 0
print(f'accuracy score: {accuracy_score(y_test, y_pred )}')
print(f'precision score: {precision_score(y_test, y_pred)}')


accuracy score: 0.6181818181818182
precision score: 1.0


In [66]:

y_pred = clf.predict([[18.,  0., 5.,  1.,  1.,  2.,  1.,  2.,  4.,  0.,  0.,  0.,  0.,
         1.,  2.,  1.,  0.,  0.,  0.,  0.,  0.,  2.,  50.,  2., 0.,  1.]])
print(y_pred)

[1]
